In [ ]:
pip install requests

In [ ]:
pip install yt-dlp

In [ ]:
pip install yt-dlp

In [ ]:
yt-dlp "https://www.youtube.com/watch?v=C253MjavrBs"

In [ ]:
import os
import subprocess
import yt_dlp
def download_youtube_video(youtube_link, output_path):
    # Extract base filename without extension
    output_base = os.path.splitext(output_path)[0]

    # Define the yt-dlp command with the necessary options
    command = [
        'yt-dlp',
        '-f', 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        youtube_link,
        '--username', 'oauth2',
        '--password', '',
        '--output', f'{output_base}.mp4'
    ]

    # Execute the command
    try:
        result = subprocess.run(command, check=True,
                                capture_output=True, text=True)
        print("Download successful:", result.stdout)
        return f"{output_base}.mp4"  # Path to the downloaded video
    except subprocess.CalledProcessError as e:
        print("An error occurred:", e.stderr)
        return None
download_youtube_video("https://www.youtube.com/watch?v=C253MjavrBs",'lex_clips.mp4')

In [2]:
pip install requests


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os 
import yt_dlp

def download_youtube_video(youtube_link, output_path):
    output_base = os.path.splitext(output_path)[0]

    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': output_base + '.mp4',
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_link])
    actual_output = output_base + '.mp4'
    return actual_output
download_youtube_video("https://www.youtube.com/watch?v=C253MjavrBs",'lex_clips.mp4')


In [7]:
import requests

# URL of the endpoint
url = "http://127.0.0.1:5000/process_video"

# The YouTube URL and path to the local audio file
data = {
    "youtube_url": "https://www.youtube.com/watch?v=C253MjavrBs"
}

# Path to the local reference audio file
files = {
    "reference_audio": open("lex_voice.wav", "rb")
}

# Make the POST request
response = requests.post(url, data=data, files=files)

# Print response from the server
print(response.status_code)
print(response.json())


200
{'matching_speakers': ['SPEAKER_01'], 'speaker_distances': {'SPEAKER_00': 0.7720759629429036, 'SPEAKER_01': 0.04126668025657687}, 'status': 'success', 'video_url': 'https://krmu-app.s3.amazonaws.com/processed_videos/output_70365d2a-222b-4630-a185-ce0f14a63bcb.mp4'}


# using pytube

In [ ]:
from flask import Flask, request, jsonify
from pytube import YouTube
import re


def download_video(url, resolution):
    try:
        yt = YouTube(url)
        stream = yt.streams.filter(progressive=True, file_extension='mp4', resolution=resolution).first()
        if stream:
            stream.download()
            return True, None
        else:
            return False, "Video with the specified resolution not found."
    except Exception as e:
        return False, str(e)

def get_video_info(url):
    try:
        yt = YouTube(url)
        stream = yt.streams.first()
        video_info = {
            "title": yt.title,
            "author": yt.author,
            "length": yt.length,
            "views": yt.views,
            "description": yt.description,
            "publish_date": yt.publish_date,
        }
        return video_info, None
    except Exception as e:
        return None, str(e)

def is_valid_youtube_url(url):
    pattern = r"^(https?://)?(www\.)?youtube\.com/watch\?v=[\w-]+(&\S*)?$"
    return re.match(pattern, url) is not None

@app.route('/download/<resolution>', methods=['POST'])
def download_by_resolution(resolution):
    data = request.get_json()
    url = data.get('url')
    
    if not url:
        return jsonify({"error": "Missing 'url' parameter in the request body."}), 400

    if not is_valid_youtube_url(url):
        return jsonify({"error": "Invalid YouTube URL."}), 400
    
    success, error_message = download_video(url, resolution)
    
    if success:
        return jsonify({"message": f"Video with resolution {resolution} downloaded successfully."}), 200
    else:
        return jsonify({"error": error_message}), 500

@app.route('/video_info', methods=['POST'])
def video_info():
    data = request.get_json()
    url = data.get('url')
    
    if not url:
        return jsonify({"error": "Missing 'url' parameter in the request body."}), 400

    if not is_valid_youtube_url(url):
        return jsonify({"error": "Invalid YouTube URL."}), 400
    
    video_info, error_message = get_video_info(url)
    
    if video_info:
        return jsonify(video_info), 200
    else:
        return jsonify({"error": error_message}), 500

